In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/data

Mounted at /content/drive
/content/drive/MyDrive/data


In [27]:
!pip install transformers -qq

     |████████████████████████████████| 5.8 MB 31.7 MB/s 
     |████████████████████████████████| 7.6 MB 83.3 MB/s 


In [28]:
import os
import tqdm
import pandas as pd
import torch
import datasets

from glob import glob
from tqdm import tqdm

from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding

In [2]:
train = pd.read_csv("train.csv")

In [66]:
test = pd.read_csv("test.csv")

In [3]:
train.head()

,ID,Utterance,Speaker,Dialogue_ID,Target
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,neutral
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,neutral
2,TRAIN_0002,That I did. That I did.,Chandler,0,neutral
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,neutral
4,TRAIN_0004,My duties? All right.,Chandler,0,surprise


# 대화별 문장

In [70]:
len(test['Dialogue_ID'].drop_duplicates())

280

In [65]:
str(train[train['Dialogue_ID'] == 0]['Utterance'].values)[2:-2]

'Why do all the coffee cups have figures below?\'\n \'Oh. It\\\'s so Monica can follow. Of this way, if one on them is missing, she can say to herself: "Where is the number 27?"\'\n \'You know what?'

In [71]:
t = []
i = 0

for i in range(len(test['Dialogue_ID'].drop_duplicates())):
  t.append(str(test[test['Dialogue_ID'] == i]['Utterance'].values)[2:-2])

In [68]:
t[:2]

['Why do all the coffee cups have figures below?\'\n \'Oh. It\\\'s so Monica can follow. Of this way, if one on them is missing, she can say to herself: "Where is the number 27?"\'\n \'You know what?',
 'Come on, Lydia, you can do it.\' \'To push!\'\n \'Push them, push them, harder, harder.\' \'Push them, push them, far!\'\n "Let\'s take this ball and really move, hey, hey, ho, ho."\n "Let\'s give up - I was right - yeah, no." \'To push!\' \'To push!']

In [72]:
df_conversation = pd.DataFrame(t, columns = ['Utterance'])
df_conversation = df_conversation.reset_index().rename(columns = {"index":"Dialogue_ID"})
df_conversation

,Dialogue_ID,Utterance
0,0,Why do all the coffee cups have figures below?...
1,1,"Come on, Lydia, you can do it.' 'To push!'\n '..."
2,2,"OK.' ""Ross, you didn't say there was an elevat..."
3,3,"Oh, okay, I understand.'\n ""No waiting, look! ..."
4,4,Ohh!' 'Quoi?' 'He kicked! I think the baby lau...
...,...,...
275,275,"Good.' 'Good.' 'Good.'\n ""Okay. We're going to..."
276,276,Hose.
277,277,What is the problem?
278,278,Someone in the metro licked my neck! I licked ...


In [75]:
test[2590:]

,ID,Utterance,Speaker,Dialogue_ID
2590,TEST_2590,It's a good idea! I bet they have one of these...,Mornica,278
2591,TEST_2591,"Yeah it's great! Besides that, Chandler will n...",Phoebe,278
2592,TEST_2592,Chandler what do you say?,Mornica,278
2593,TEST_2593,"Okay, but I should warn you, I'm not going. I’...",Chadler,278
2594,TEST_2594,Give them!,Mornica,279
2595,TEST_2595,No! They are mine!,Rachell,279
2596,TEST_2596,You stole them from me!,Mornica,279
2597,TEST_2597,You stole them !!,Rachell,279
2598,TEST_2598,Give them!,Mornica,279
2599,TEST_2599,Do you just want to take one?,Mornica,279


In [76]:
df_test = pd.merge(test,df_conversation, on = 'Dialogue_ID', how = 'left')

In [77]:
df_test

,ID,Utterance_x,Speaker,Dialogue_ID,Utterance_y
0,TEST_0000,Why do all the coffee cups have figures below?,Mark,0,Why do all the coffee cups have figures below?...
1,TEST_0001,"Oh. It's so Monica can follow. Of this way, if...",Rachell,0,Why do all the coffee cups have figures below?...
2,TEST_0002,You know what?,Rachell,0,Why do all the coffee cups have figures below?...
3,TEST_0003,"Come on, Lydia, you can do it.",Joeyy,1,"Come on, Lydia, you can do it.' 'To push!'\n '..."
4,TEST_0004,To push!,Joeyy,1,"Come on, Lydia, you can do it.' 'To push!'\n '..."
...,...,...,...,...,...
2605,TEST_2605,"Yeah, I mean, go Ross, no one will even notice...",Rachell,279,Give them!' 'No! They are mine!' 'You stole th...
2606,TEST_2606,They don't listen to me?,Rossi,279,Give them!' 'No! They are mine!' 'You stole th...
2607,TEST_2607,"Of course, they listen to you! Everyone listen...",Rachell,279,Give them!' 'No! They are mine!' 'You stole th...
2608,TEST_2608,"Monica, do you really think I should try this ...",Rossi,279,Give them!' 'No! They are mine!' 'You stole th...


In [21]:
df_train['Target'] = df_train['Target'].map({'neutral':0,
                                 'joy':1,
                                 'surprise':2,
                                 'anger':3,
                                 'sadness':4,
                                 'disgust':5,
                                 'fear':6})

df_train.head()

,ID,Utterance_x,Speaker,Dialogue_ID,Target,Utterance_y
0,TRAIN_0000,also I was the point person on my company’s tr...,Chandler,0,0,also I was the point person on my company’s tr...
1,TRAIN_0001,You must’ve had your hands full.,The Interviewer,0,0,also I was the point person on my company’s tr...
2,TRAIN_0002,That I did. That I did.,Chandler,0,0,also I was the point person on my company’s tr...
3,TRAIN_0003,So let’s talk a little bit about your duties.,The Interviewer,0,0,also I was the point person on my company’s tr...
4,TRAIN_0004,My duties? All right.,Chandler,0,2,also I was the point person on my company’s tr...


In [22]:
from sklearn.model_selection import train_test_split

t_data, v_data = train_test_split(df_train, test_size = 0.2, random_state = 42,
                                  shuffle = True)

In [23]:
test = pd.read_csv("test.csv")

In [16]:
!pip install datasets -qq

     |████████████████████████████████| 451 kB 29.4 MB/s 
     |████████████████████████████████| 182 kB 98.6 MB/s 
     |████████████████████████████████| 212 kB 98.8 MB/s 
     |████████████████████████████████| 132 kB 94.4 MB/s 
     |████████████████████████████████| 127 kB 100.8 MB/s 


In [25]:
from datasets import Dataset
import datasets

train_dataset = Dataset.from_pandas(t_data)
val_dataset = Dataset.from_pandas(v_data)
test_dataset = Dataset.from_pandas(test)

raw_dataset = datasets.DatasetDict({"train":Dataset.from_dict({'text': train_dataset['Utterance_x'], 'labels':train_dataset['Target']}), 
                                    "valid":Dataset.from_dict({'text': val_dataset['Utterance_x'], 'labels':val_dataset['Target']}), 
                                    "test":Dataset.from_dict({'text': test_dataset['Utterance']})})
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 7991
    })
    valid: Dataset({
        features: ['text', 'labels'],
        num_rows: 1998
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2610
    })
})

In [26]:
model_name = 'tae898/emoberta-large'

batch_size = 16
learning_rate = 5e-5
num_epochs = 4

In [29]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding = True, max_length = 128)

tokenized_datasets = raw_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/408 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [30]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 7991
    })
    valid: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1998
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 2610
    })
})

In [31]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [32]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)
valid_dataloader = DataLoader(tokenized_datasets["valid"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_datasets["test"], shuffle=False, batch_size=batch_size, collate_fn=data_collator)

In [33]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 61]),
 'attention_mask': torch.Size([16, 61])}

In [34]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7)

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [35]:
from transformers import get_scheduler, AdamW
optimizer = AdamW(model.parameters(), lr=learning_rate)

num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2000


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [37]:
from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)


  0%|          | 0/2000 [00:00<?, ?it/s]

In [38]:
!pip install torchmetrics -qq

     |████████████████████████████████| 512 kB 30.1 MB/s 


In [41]:
prediction_list_valid = []
target_list_valid = []

model.eval()
for batch in valid_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=0).cpu()
    targets = batch['labels'].cpu()

    prediction_list_valid.extend(predictions)
    target_list_valid.extend(targets)
    #print(accuracy(predictions, targets)) # 매 batch 마다의 Accuracy

In [42]:
# 원래꺼

prediction_list = []
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=0)
    prediction_list.extend(logits.cpu())

In [49]:
prediction_list[:5]

[tensor([ 2.2847, -1.1348,  2.9768, -0.4259, -0.5553, -2.3839, -1.1369]),
 tensor([ 5.7149,  0.0183, -1.4100, -1.5259, -1.0635, -2.2816, -1.3435]),
 tensor([ 4.2439,  0.1277, -0.9537,  0.2987, -0.9476, -1.6535, -1.7580]),
 tensor([ 4.9757,  2.0290, -2.1188, -1.9425, -1.0572, -3.3914,  0.0367]),
 tensor([ 2.2847,  2.6658, -0.6372, -0.6349, -1.2499, -2.0087, -1.0609])]

# dialogue

In [78]:
di_dataset = Dataset.from_pandas(df_test)

di_dataset =  datasets.DatasetDict({"Dialogue":Dataset.from_dict({'text': di_dataset['Utterance_y']})})
di_dataset

DatasetDict({
    Dialogue: Dataset({
        features: ['text'],
        num_rows: 2610
    })
})

In [79]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding = True, max_length = 128)

di_tokenized_datasets = di_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [80]:
di_tokenized_datasets

DatasetDict({
    Dialogue: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 2610
    })
})

In [81]:
di_tokenized_datasets = di_tokenized_datasets.remove_columns(["text"])
di_tokenized_datasets["Dialogue"].column_names

['input_ids', 'attention_mask']

In [82]:
from torch.utils.data import DataLoader

di_dataloader = DataLoader(di_tokenized_datasets["Dialogue"], shuffle=True, batch_size=batch_size, collate_fn=data_collator)

for batch in di_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': torch.Size([16, 128]), 'attention_mask': torch.Size([16, 128])}

In [83]:
di_prediction_list = []
model.eval()
for batch in di_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=0)
    di_prediction_list.extend(logits.cpu())

In [60]:
len(prediction_list)

2610

In [85]:
len(di_prediction_list)

2610

In [107]:
final_list = []

for i in range(len(prediction_list)):
  final_list.append(torch.cat([prediction_list[i],di_prediction_list[i]], dim = 0))

In [108]:
final_list[:5]

[tensor([ 2.2847, -1.1348,  2.9768, -0.4259, -0.5553, -2.3839, -1.1369, -2.2901,
          0.5197,  2.3386,  1.2294,  0.3879,  1.2293, -1.5974]),
 tensor([ 5.7149e+00,  1.8260e-02, -1.4100e+00, -1.5259e+00, -1.0635e+00,
         -2.2816e+00, -1.3435e+00,  3.4521e+00,  6.9030e-01, -1.3827e+00,
          2.5160e-01, -1.9670e-03, -2.0674e+00, -1.5925e+00]),
 tensor([ 4.2439,  0.1277, -0.9537,  0.2987, -0.9476, -1.6535, -1.7580,  1.4534,
          3.3022, -1.2247,  0.9199, -0.1568, -2.8385, -1.2967]),
 tensor([ 4.9757,  2.0290, -2.1188, -1.9425, -1.0572, -3.3914,  0.0367, -0.9981,
         -0.9021,  0.8684,  1.6940,  1.7726, -1.1762,  0.8158]),
 tensor([ 2.2847,  2.6658, -0.6372, -0.6349, -1.2499, -2.0087, -1.0609, -0.2888,
          2.8920, -1.7013,  1.8910,  1.0186, -1.2494, -1.6889])]

In [110]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [142]:
final_list[0].numpy().shape

(14,)

In [148]:
final_list[0]

tensor([ 2.2847, -1.1348,  2.9768, -0.4259, -0.5553, -2.3839, -1.1369, -2.2901,
         0.5197,  2.3386,  1.2294,  0.3879,  1.2293, -1.5974])

In [226]:
m = nn.Linear(14,7)
m(final_list[0])

tensor([-2.0535, -0.3110,  0.1094,  0.9164,  0.2454, -1.6638, -1.4886],
       grad_fn=<AddBackward0>)

In [228]:
y_pred = []

for i in range(len(final_list)):
  y_pred.append(np.argmax(m(final_list[i]).detach().cpu().numpy()))

In [230]:
set(y_pred)

{0, 1, 2, 3, 4, 5, 6}

In [232]:
from collections import Counter

Counter(y_pred).values()

dict_values([104, 1695, 78, 254, 223, 44, 212])

In [172]:
sample = pd.read_csv("sample_submission.csv")
sample.head()

,ID,Target
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0


In [240]:
sample['Target'] = y_pred

In [241]:
sample['Target'] = sample['Target'].map({
                                 0 :'joy',
                                 1: 'surprise',
                                 2: 'neutral',
                                 3: 'sadness',
                                 4: 'disgust',
                                 5: 'fear',
                                 6: 'anger'})

sample.head()

,ID,Target
0,TEST_0000,sadness
1,TEST_0001,neutral
2,TEST_0002,neutral
3,TEST_0003,neutral
4,TEST_0004,neutral


In [242]:
sample['Target'].value_counts()

neutral     1695
joy          254
surprise     223
anger        212
sadness      104
disgust       78
fear          44
Name: Target, dtype: int64

In [243]:
sample.to_csv('our_fucking.csv', index = False)